# Overview

The generalization performance of a learning method relates to its prediction
capability on independent test data. Assessment of this performance
is extremely important in practice, since it guides the choice of learning
method or model, and gives us a measure of the quality of the ultimately
chosen model


# Cross-validation

## Validation Set Approach

It involves randomly dividing the available set of observations into two parts, a training set and a validation set or hold-out set. The model is fit on the training set, and the fitted model is used to predict the responses for the observations in the validation set. The resulting validation set error rate provides an estimate of the test error rate. 

<img src="validation_set_approach.png" alt="Drawing" style="width: 500px;"/>

In [1]:
import pandas as pd 
import numpy as np
from sklearn import datasets, linear_model
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

%matplotlib inline

data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data-original",
                   delim_whitespace = True, header=None,
                   names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
                            'model', 'origin', 'car_name'])
data = data.dropna(axis = 0)
data.reset_index(drop=True, inplace=True)
data.describe()

/home/vasilis/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,1.576531
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.805518
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,1.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


In [2]:
X = data['horsepower'].reshape(-1, 1) # We need to reshape the array so that we can feed it to the model
y = data['mpg']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Simple linear regression
lm = linear_model.LinearRegression()
lm.fit(x_train, y_train)
lm_pred = lm.predict(x_test)

np.mean((y_test - lm_pred)**2)


/home/vasilis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


25.57387818968439

In [3]:
# Let's try quadratic regression
X_2 = np.hstack((X, X**2)) 
x_train, x_test, y_train, y_test = train_test_split(X_2, y, test_size=0.5, random_state=42)

lm = linear_model.LinearRegression()
lm.fit(x_train, y_train)
lm_pred = lm.predict(x_test)
np.mean((y_test - lm_pred)**2)


22.218020050032855

In [4]:
# Let's try cubic regression
X_3 = np.hstack((X, X**2, X**3))
x_train, x_test, y_train, y_test = train_test_split(X_3, y, test_size=0.5, random_state=42)

lm = linear_model.LinearRegression()
lm.fit(x_train, y_train)
lm_pred = lm.predict(x_test)
np.mean((y_test - lm_pred)**2)

22.667675435534484

Let's change the *random state* value to see if we observe any difference. 

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=41)
# Simple linear regression
lm = linear_model.LinearRegression()
lm.fit(x_train, y_train)
lm_pred = lm.predict(x_test)

print(np.mean((y_test - lm_pred)**2))

# Quadratic regression
x_train, x_test, y_train, y_test = train_test_split(X_2, y, test_size=0.5, random_state=41)

lm = linear_model.LinearRegression()
lm.fit(x_train, y_train)
lm_pred = lm.predict(x_test)

print(np.mean((y_test - lm_pred)**2))

x_train, x_test, y_train, y_test = train_test_split(X_3, y, test_size=0.5, random_state=41)
lm = linear_model.LinearRegression()
lm.fit(x_train, y_train)
lm_pred = lm.predict(x_test)

print(np.mean((y_test - lm_pred)**2))


25.41900883210308
20.19623577421668
20.26642632189447


We can see that non linear methods perform much better. But also we notice that this way of validating is really not consistent. 

## Leave-One-Out Cross-Validation Approach

It is very similar to **Validation Set Approach** but it attempts to address this method's drawbacks. 

It is also separates the whole data set into training and validation set, but instead of creating two data sets of comparing size a single observation is used for validation set. This process takes place up *n* times so that we have used all the observations as a validation set. 


<img src="loocv.png" alt="Drawing" style="width: 500px;"/>

The estimate of test error from this method is quite poor because it will be quite variable since it is based on only one observation.

Let's assume that we use as a loss function the MSE. The LOOCV estimate for test error is: 
$$CV = \dfrac{1}{n}\sum_{i}^{n}MSE_{i} $$ 

Why do we use LOOCV approach over the Validation Set approach?

* Less bias, the LOOCV approach doesn't overestimate the test set as Validation Set approach does.
* Validation set approach will return different results if you run it multiple times due to the randomness of set selection but LOOCV will return the same results

LOOCV can be very expensive if we have a big data set. Is there another cheaper way to do that? Of course there is!



In [6]:
loo = LeaveOneOut()
error = []
lm = linear_model.LinearRegression()
for train_index, test_index in loo.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm.fit(x_train, y_train)
    lm_pred = lm.predict(x_test)
    error.append((np.mean((y_test - lm_pred)**2)))

print("The average error is:",np.mean(error), "The standard deviation is:",np.std(error))

The average error is: 24.2315135179 The standard deviation is: 36.7973150364


In [7]:
loo = LeaveOneOut()
error = []
lm = linear_model.LinearRegression()
for train_index, test_index in loo.split(X):
    x_train, x_test = X_2[train_index], X_2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm.fit(x_train, y_train)
    lm_pred = lm.predict(x_test)
    error.append((np.mean((y_test - lm_pred)**2)))

print("The average error is:",np.mean(error), "The standard deviation is:",np.std(error))

The average error is: 19.2482131245 The standard deviation is: 34.9984461518


In [8]:
loo = LeaveOneOut()
error = []
lm = linear_model.LinearRegression()
for train_index, test_index in loo.split(X):
    x_train, x_test = X_3[train_index], X_3[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm.fit(x_train, y_train)
    lm_pred = lm.predict(x_test)
    error.append((np.mean((y_test - lm_pred)**2)))

print("The average error is:",np.mean(error), "The standard deviation is:",np.std(error))

The average error is: 19.334984064 The standard deviation is: 35.7651356779


You can see that the estimation of the error is much more accurate

## $k$-Fold Cross Validation approach

This approach divides the original set into $k$ groups(aka folds) of equal size. The first fold is treated as validation set and the method is fitted into the rest $k-1$ folds. This process, results into $k$ estimates.

<img src="kfolds.png" alt="Drawing" style="width: 500px;"/>


Let's assume that we use as a loss function the MSE. The $k$-Fold CV estimate for test error is: 

$$CV = \dfrac{1}{k}\sum_{i}^{k}MSE_{i}$$  

Why do we use k-Fold CV instead of LOOCV?

* It is much faster
* Bias-Variance trade-off, $k$-Fold CV gives better test estimates because:
    * Validation set approach overestimates the test error and LOOCV is quite unbiased while $k$-Fold is in between. When we consider to minimize bias we go for LOOCV, but bias is not the only source for concern of estimation;
    * LOOCV has higher variance than $k$-Fold CV 

In [9]:
k_fold = KFold(n_splits = 10)
error = []
lm = linear_model.LinearRegression()
for train_index, test_index in k_fold.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm.fit(x_train, y_train)
    lm_pred = lm.predict(x_test)
    error.append((np.mean((y_test - lm_pred)**2)))

print("The average error is:",np.mean(error), "The standard deviation is:",np.std(error))

The average error is: 27.4399336523 The standard deviation is: 14.5102507113


In [10]:
k_fold = KFold(n_splits = 10)
error = [] 
lm = linear_model.LinearRegression()
for train_index, test_index in k_fold.split(X_2):
    x_train, x_test = X_2[train_index], X_2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm.fit(x_train, y_train)
    lm_pred = lm.predict(x_test)
    error.append((np.mean((y_test - lm_pred)**2)))

print("The average error is:",np.mean(error), "The standard deviation is:",np.std(error))

The average error is: 21.2358400558 The standard deviation is: 11.7973275289


In [11]:
k_fold = KFold(n_splits = 10)
error = []
lm = linear_model.LinearRegression()
for train_index, test_index in k_fold.split(X_3):
    x_train, x_test = X_3[train_index], X_3[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm.fit(x_train, y_train)
    lm_pred = lm.predict(x_test)
    error.append((np.mean((y_test - lm_pred)**2)))

print("The average error is:",np.mean(error), "The standard deviation is:",np.std(error))

The average error is: 21.3366061832 The standard deviation is: 11.8443397146


Notice that K-Fold is much faster approach. Also notice the variance of the two methods. K-fold has a less variant result.

# Bootstrap

Bootstrap is a method used to quantify the uncertainty of the estimator (e.g standard errors of the coefficients in linear regression) of a statistical learning model. 


It samples randomly with replacement *m* data sets with *n* observations from the original data set. Having created the *m* data sets

<img src="bootstrapping.jpg" alt="Drawing" style="width: 500px;"/>

Having done that, you calculate the variance for every sample data set and then average it.

$$S(\bar{X}) = \dfrac{1}{m}\sum_{i=1}^{m}(S(X^{*i})) $$

The function for calculating the bootstrap method was found [here](http://nbviewer.jupyter.org/gist/aflaxman/6871948).

In [12]:
def bootstrap_resample(X, n=None):
    """ Bootstrap resample an array_like
    Parameters
    ----------
    X : array_like
      data to resample
    n : int, optional
      length of resampled array, equal to len(X) if n==None
    Results
    -------
    returns X_resamples
    """
    if n == None:
        n = len(X)
        
    resample_i = np.floor(np.random.rand(n)*len(X)).astype(int)
    X_resample = X[resample_i]
    return X_resample

for i in range(1000):
    train_index = bootstrap_resample(np.where(X)[0], n=196)
    test_index = bootstrap_resample(np.where(X)[0], n=196)
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm.fit(x_train, y_train)
    lm_pred = lm.predict(x_test)
    error.append((np.mean((y_test - lm_pred)**2)))

print("The average error is:",np.mean(error), "The standard deviation is:",np.std(error))

for i in range(1000):
    train_index = bootstrap_resample(np.where(X)[0], n=196)
    test_index = bootstrap_resample(np.where(X)[0], n=196)
    x_train, x_test = X_2[train_index], X_2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm.fit(x_train, y_train)
    lm_pred = lm.predict(x_test)
    error.append((np.mean((y_test - lm_pred)**2)))

print("The average error is:",np.mean(error), "The standard deviation is:",np.std(error))

for i in range(1000):
    train_index = bootstrap_resample(np.where(X)[0], n=196)
    test_index = bootstrap_resample(np.where(X)[0], n=196)
    x_train, x_test = X_3[train_index], X_3[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm.fit(x_train, y_train)
    lm_pred = lm.predict(x_test)
    error.append((np.mean((y_test - lm_pred)**2)))

print("The average error is:",np.mean(error), "The standard deviation is:",np.std(error))

The average error is: 24.1620693417 The standard deviation is: 2.90864286876
The average error is: 21.7092268932 The standard deviation is: 3.7026353213
The average error is: 20.9242564749 The standard deviation is: 3.54980821034
